# Processamento de Linguagens e Compiladores

## Licenciatura em Ciências da Computação

## Trabalho Prático 1 - Relatório de Desenvolvimento

André Lucena Ribas Ferreira (A94956)

Carlos Eduardo da Silva Machado (A96936)¶

Gonçalo Manuel Maia de Sousa (A97485)

## Introdução

### Resumo

O presente Relatório tem como objetivo apresentar a nossa solução para o Trabalho Prático 1 da Unidade Curricular de Processamento de Linguagens e Compiladores. Especificamente, demonstramos a conceção de duas soluções para o Problema 5 "Ficheiros CSV com listas e funções de agregação", onde se procura converter um ficheiro do tipo CSV para JSON. A primeira solução utiliza reduzidamente o módulo *re*, suportando-se no Python e nas suas ferramentas, enquanto que a segunda procura maximizar o uso de expressões regulares e das funções desse mesmo módulo.

### Estrutura

## Especificação do Problema

### Análise

### Definição dos Requisitos

## Resolução

Para resolver o Problema de acordo com os Requisitos enunciados, as seguintes linhas de código são comuns para ambos as soluções. 

Utilizaremos o módulo 're', que oferece operações de matching de expressões regulares em python.

In [1]:
import re
from pprint import pprint

De seguida, obtemos o ficheiro de input do utilizador, do tipo CSV.

In [2]:
filename = 'teste1.csv'#input("introduza o ficheiro de input:")
input = open(filename, "r", encoding="utf-8")

O primeiro passo de ambas as soluções é o de interpretar a primeira linha, de modo a conseguir uma caracterização das colunas que fazem parte do ficheiro. Para tal, construimos uma expressão regular que, com a utilização de grupos e da interação da função *findall* com esses mesmos grupos, nos devolve uma lista que define completamente cada uma das colunas.

Mais precisamente, cada entrada da lista é um tuplo com 4 componentes, todas do tipo *string*. A presença dos três últimos é facultativa e indica se a coluna tem elemento mínimo, se forma uma lista e qual o seu elemento máximo, e se tem uma função aplicada.
 1. Título; 
 2. Número mínimo de elementos da lista; 
 3. Número máximo de elementos da lista; 
 4. Função aplicada à lista.

A expressão regular começa por definir uma palavra obrigatória **([^{,]+)** , capturada no primeiro grupo, que é o Título da coluna. Nesta parte, utiliza-se parênteses retos com acento circunflexo para denotar qualquer letra que não as especificadas, nomeadamente chaveta esquerda e vírgula, e depois um operador para repetir, pelo menos uma vez, essas letras.

De seguida, definimos dois blocos facultativos, utilizando o operador **(?:)** para não capturar os grupos definidos pelos parênteses necessários para o operador **?** que os torna opcionais. O primeiro representa as listas, onde pelo menos um número é garantido, definido de tal modo a pertencer ao grupo 3, capturando no grupo 2 o valor mínimo se existente. O segundo representa a função, cuja designação encontra-se capturada no grupo 4.

In [3]:
primeira = input.readline()[:-1]
colunas = re.findall(r'([^{,]+)(?:{(?:(\d+),)?(\d+)})?(?:::([^,]+))?', primeira)
pprint(colunas)

[('Número', '', '', ''),
 ('Nome', '', '', ''),
 ('Curso', '', '', ''),
 ('Notas', '', '5', 'sum'),
 ('Pokemon', '', '', ''),
 ('ListaSadakaNoFun', '2', '3', '')]


### Solução 2

In [4]:
patern = fun_str = r''
replace = '\t{\n'

In [5]:
#colunas: 0 -> título; 1->min/único; 2->max; 3->função
for i,coluna in enumerate(colunas, start=1):
    if coluna[2] == '':
        patern += r'([^{,]+),'    
        replace += '\t\t"%s": "\%d",\n' % (coluna[0],i)
    else:
        max = int(coluna[2])
        min = max if coluna[1] == '' else int(coluna[1])
        patern += r'([^,]+(?:,[^,]+){%d,%d}),{0,%d},' % (min-1, max-1, max - min)
        if coluna[3] != '':
            fun_str += r'%s|' % (coluna[3])
            replace += '\t\t"%s_%s": %s([\%d]),\n' % (coluna[0],coluna[3],coluna[3],i)
        else:
            replace += '\t\t"%s": [\%d],\n' % (coluna[0],i)

print(patern)

([^{,]+),([^{,]+),([^{,]+),([^,]+(?:,[^,]+){4,4}),{0,0},([^{,]+),([^,]+(?:,[^,]+){1,2}),{0,1},


In [6]:
patern = patern[:-1]
replace = re.sub(r',\n$', '\n\t},\n', replace) #replace = replace[:-2] + '\n\t},\n'
fun_str = fun_str[:-1]

In [7]:
res = '[\n'
for linha in input:
    linha = linha[:-1]
    #print(linha)
    res += re.sub(patern,replace,linha)

res = re.sub(r'(((?:%s))\(.*\))'%(fun_str), lambda x: str(eval(x.group(1))),res)
res = re.sub(r',\n$', '\n]', res)#res = res[:-2] + "\n]"

In [8]:
input.close()

filename = re.sub(r'csv',r'json',filename)

output = open(filename, 'w', encoding="utf-8")

print(res) #remover depois ou não 

output.write(res)
output.close()

[
	{
		"Número": "3162",
		"Nome": "Cândido Faísca",
		"Curso": "Teatro",
		"Notas_sum": 50,
		"Pokemon": "Giratina",
		"ListaSadakaNoFun": [1,2]
	},
	{
		"Número": "7777",
		"Nome": "Cristiano Ronaldo",
		"Curso": "Desporto",
		"Notas_sum": 46,
		"Pokemon": "Garbodor",
		"ListaSadakaNoFun": [3,4,5]
	},
	{
		"Número": "264",
		"Nome": "Marcelo Sousa",
		"Curso": "Ciência Política",
		"Notas_sum": 42,
		"Pokemon": "Vanilux",
		"ListaSadakaNoFun": [5,6]
	}
]


### Solução 2